<h1>Proyecto Tecnologías de Búsqueda en la Web</h1>
<h3>Integrantes</h3>
<ul><li>Sebastián Aranda</li><li>Felipe Santander</li></ul>

<h2>Librerías</h2>

In [2]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary, bleicorpus
from gensim.models import ldamodel
from gensim.models import Phrases

from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import pyLDAvis
import os

<h2>Creación del Corpus</h2>

In [3]:
corpus_path = "cyberbullying_corpus/"
#corpus_path = "corpus_lda/corpus_lda.lda_c"
#dictionary_path ="corpus_lda/corpus_lda.dict"

In [9]:
#Remove keywords in cyberbullying from stopwords list
keywords = ["you", "your", "he", "she", "it"]
stopword_list = [stopword for stopword in stopwords.words('english') if stopword not in keywords]
puncts = ".,:;?!()[]{}~+-\"\'#$%&\/"
digits = "0123456789"

#Create the corpus
corpus = []
for filename in os.listdir(corpus_path):
    with open(corpus_path+filename) as file:
        tokens = []
        for line in file:
            for sym in puncts:
                line = line.replace(sym," ")
            for num in digits:
                line = line.replace(num," ")

            tokens = tokens + [word for word in line.lower().split() if word not in stopword_list]
        corpus.append(tokens)

#Remove words that appear only once
frequency = defaultdict(int)
for text in corpus:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1] for text in corpus]


# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
texts = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts]

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
#bigram = Phrases(texts, min_count=20)
#for idx in range(len(texts)):
#    for token in bigram[texts[idx]]:
#        if '_' in token:
#            texts[idx].append(token)

#Create and save dictionary
dictionary = corpora.Dictionary(texts)
dictionary.save('tmp/cyberbullying_dictionary.dict') #Save the dictionary

#Convert documents to vectors
corpus = [dictionary.doc2bow(text) for text in texts]

#Increasing weight of bad words:
bad_words = ["ass", "face", "dick", "asshole"]
bad_words_ids = [word_id for word_id, word in dictionary.iteritems() if word in bad_words]

for doc_idx in range(len(corpus)):
    word_id_list = []
    freq_list = []

    for word_id, freq in corpus[doc_idx]:
        word_id_list.append(word_id)
        if word_id in bad_words_ids:
            freq_list.append(freq*2000)
        else:
            freq_list.append(freq)
    
    from random import randint
    if randint(0, 9) > 8:
        word_id_list.append(bad_words_ids[0])
        freq_list.append(5000)
    
    new_doc = zip(word_id_list,freq_list)
    corpus[doc_idx] = new_doc
    
    #print("Most frequent word of Doc_"+str(doc_idx))
    #print(dictionary[word_id_list[freq_list.index(max(freq_list))]])
    #print("----------------------------\n")

corpora.BleiCorpus.serialize('tmp/cyberbullying_corpus.lda-c', corpus) #Save the corpus


In [ ]:
#Load the dictionary and corpus
if (os.path.exists('tmp/cyberbullying_dictionary.dict') and os.path.exists('tmp/cyberbullying_corpus.lda-c')):
    print('Creating dictionary...')
    %time dictionary = corpora.Dictionary.load('tmp/cyberbullying_dictionary.dict')
    print('Creating corpus...')
    %time corpus = corpora.BleiCorpus('tmp/cyberbullying_corpus.lda-c')
else:
    print("Create the dictionary and corpus first")

In [10]:
#Initialize a model
#print('Creating Tfidf model...')
#%time tfidf = models.TfidfModel(corpus)
#corpus_tfidf = tfidf[corpus]

#Create a LDA Model and save it
lda = ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary)
lda.save('tmp/cyberbullying_ldaModel.lda')

In [ ]:
#Load LDA Model
lda = models.LdaModel.load('tmp/cyberbullying_ldaModel.lda')

In [13]:
#Print Topics
lda.print_topics(2,100)

[(0,
  u'0.780*"face" + 0.163*"dick" + 0.009*"asshole" + 0.004*"you" + 0.003*"ass" + 0.002*"your" + 0.001*"like" + 0.001*"look" + 0.001*"it" + 0.000*"he" + 0.000*"get" + 0.000*"one" + 0.000*"guy" + 0.000*"would" + 0.000*"make" + 0.000*"she" + 0.000*"know" + 0.000*"hair" + 0.000*"girl" + 0.000*"kid" + 0.000*"think" + 0.000*"time" + 0.000*"even" + 0.000*"see" + 0.000*"say" + 0.000*"roast" + 0.000*"got" + 0.000*"head" + 0.000*"go" + 0.000*"could" + 0.000*"going" + 0.000*"need" + 0.000*"want" + 0.000*"people" + 0.000*"life" + 0.000*"year" + 0.000*"http" + 0.000*"bet" + 0.000*"u" + 0.000*"eye" + 0.000*"back" + 0.000*"thing" + 0.000*"school" + 0.000*"picture" + 0.000*"take" + 0.000*"shit" + 0.000*"fuck" + 0.000*"someone" + 0.000*"right" + 0.000*"friend" + 0.000*"com" + 0.000*"probably" + 0.000*"tell" + 0.000*"already" + 0.000*"fucking" + 0.000*"mom" + 0.000*"good" + 0.000*"old" + 0.000*"as" + 0.000*"really" + 0.000*"way" + 0.000*"ever" + 0.000*"never" + 0.000*"still" + 0.000*"gay" + 0.000*"s